In [1]:
import tabula
import pandas as pd
import numpy as np
import requests
import warnings

warnings.filterwarnings("ignore")



URL = "https://www.tcmb.gov.tr/wps/wcm/connect/a6ffdb2f-47d9-4ae9-8c39-5075867aaec3/TCMB+Taraflı+Swap+İşlemleri.pdf?MOD=AJPERES"


def fetch_domestic_banks_fx_swap(url: str = URL) -> pd.DataFrame:
    # 1) PDF indir
    resp = requests.get(url)
    resp.raise_for_status()
    filename = "swap.pdf"
    with open(filename, "wb") as f:
        f.write(resp.content)

    # 2) Tabloları oku ve birleştir
    tables = tabula.read_pdf(filename, pages="all", multiple_tables=True)
    df = pd.concat(tables, ignore_index=True)

    # 3) İlk satırı kolon adı yap (senin yaptığın gibi)
    df.columns = df.iloc[0]
    df = df.iloc[1:].reset_index(drop=True)

    # 4) Tarihi parse et
    first_col = df.columns[0]
    df[first_col] = (
        df[first_col]
        .astype(str)
        .str.replace("\r", "", regex=False)
        .str.strip()
        .pipe(pd.to_datetime, dayfirst=True, errors="coerce")
    )

    # Tarih olmayan satırları at ve index yap
    df = df.dropna(subset=[first_col]).set_index(first_col)
    df.index.name = "Tarih"

    # 5) Sayısal kolonları temizle (rakam dışı her şeyi at, boşları 0 yap, int'e çevir)
    df = (
        df.replace(r"[^\d\-]", "", regex=True)
          .replace("", np.nan)
          .fillna(0)
          .astype(int)
    )

    # 6) Sadece TOPLAM - STOK TOTAL OUTSTANDING kolonlarını bul
    cols = df.columns.to_series()

    total_mask = (
        cols.str.contains("TOPLAM", case=False, na=False)
        & cols.str.contains("OUTSTANDING", case=False, na=False)
    )

    total_cols = cols[total_mask].tolist()

    # Beklediğimiz: 2 kolon (Alım / Satım)
    # İsimlerden programatik olarak Buy/Sell ayıralım
    col_buy = [c for c in total_cols if ("Alım" in c) or ("Buy" in c)][0]
    col_sell = [c for c in total_cols if ("Satım" in c) or ("Sell" in c)][0]

    # 7) Swap serisini hesapla
    out = pd.DataFrame(index=df.index)
    out["swap"] = df[col_buy] - df[col_sell]
    out.index.name = "Tarih"

    return out


# Kullanım
df = fetch_domestic_banks_fx_swap()
df.to_excel("results/domestic_banks_fx_swap.xlsx")
df

Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'


,swap
Tarih,
2021-01-04,44039
2021-01-05,43782
2021-01-06,43683
2021-01-07,43500
2021-01-08,43601
...,...
2025-12-08,-2430
2025-12-09,-3027
2025-12-10,-3963
